# <center> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Identify Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score
import pickle

**Считаем данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
train_df = pd.read_csv('kaggle_data/train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('kaggle_data/test_sessions.csv', index_col='session_id')

In [3]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,8,2014-01-04 08:44:50,11.0,2014-01-04 08:44:50,82.0,2014-01-04 08:45:19,68.0,2014-01-04 08:45:25,8.0,2014-01-04 08:45:25,...,2014-01-04 08:45:51,8403.0,2014-01-04 08:45:51,932.0,2014-01-04 08:45:53,3260.0,2014-01-04 08:45:53,8.0,2014-01-04 08:45:53,1845
2,111,2014-03-18 10:33:20,78.0,2014-03-18 10:33:31,151.0,2014-03-18 10:33:31,111.0,2014-03-18 10:33:31,1401.0,2014-03-18 10:33:31,...,2014-03-18 10:33:32,1375.0,2014-03-18 10:33:32,38.0,2014-03-18 10:33:32,1401.0,2014-03-18 10:33:32,97.0,2014-03-18 10:33:34,3322
3,11,2014-12-02 13:13:41,3187.0,2014-12-02 13:13:41,132.0,2014-12-02 13:13:42,496.0,2014-12-02 13:13:42,1969.0,2014-12-02 13:13:45,...,2014-12-02 13:13:45,3187.0,2014-12-02 13:13:45,82.0,2014-12-02 13:13:46,3191.0,2014-12-02 13:13:46,3184.0,2014-12-02 13:13:47,2003
4,668,2014-02-14 15:16:45,1965.0,2014-02-14 15:17:13,598.0,2014-02-14 15:20:47,1965.0,2014-02-14 15:21:13,284.0,2014-02-14 15:21:14,...,2014-02-14 15:21:14,38.0,2014-02-14 15:21:14,4451.0,2014-02-14 15:21:14,4537.0,2014-02-14 15:21:15,11.0,2014-02-14 15:21:15,1373
5,1943,2014-03-17 15:19:40,1943.0,2014-03-17 15:20:10,1943.0,2014-03-17 15:21:40,1943.0,2014-03-17 15:22:10,1943.0,2014-03-17 15:22:39,...,2014-03-17 15:22:39,1952.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:42,1943.0,2014-03-17 15:22:43,1737


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [4]:
train_test_df = pd.concat([train_df, test_df])

In [5]:
train_test_df.head()

,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,time10,time2,time3,time4,time5,time6,time7,time8,time9,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,8,8.0,11.0,82.0,68.0,8.0,8393.0,8403.0,932.0,3260.0,...,2014-01-04 08:45:53,2014-01-04 08:44:50,2014-01-04 08:45:19,2014-01-04 08:45:25,2014-01-04 08:45:25,2014-01-04 08:45:51,2014-01-04 08:45:51,2014-01-04 08:45:53,2014-01-04 08:45:53,1845.0
2,111,97.0,78.0,151.0,111.0,1401.0,151.0,1375.0,38.0,1401.0,...,2014-03-18 10:33:34,2014-03-18 10:33:31,2014-03-18 10:33:31,2014-03-18 10:33:31,2014-03-18 10:33:31,2014-03-18 10:33:32,2014-03-18 10:33:32,2014-03-18 10:33:32,2014-03-18 10:33:32,3322.0
3,11,3184.0,3187.0,132.0,496.0,1969.0,504.0,3187.0,82.0,3191.0,...,2014-12-02 13:13:47,2014-12-02 13:13:41,2014-12-02 13:13:42,2014-12-02 13:13:42,2014-12-02 13:13:45,2014-12-02 13:13:45,2014-12-02 13:13:45,2014-12-02 13:13:46,2014-12-02 13:13:46,2003.0
4,668,11.0,1965.0,598.0,1965.0,284.0,668.0,38.0,4451.0,4537.0,...,2014-02-14 15:21:15,2014-02-14 15:17:13,2014-02-14 15:20:47,2014-02-14 15:21:13,2014-02-14 15:21:14,2014-02-14 15:21:14,2014-02-14 15:21:14,2014-02-14 15:21:14,2014-02-14 15:21:15,1373.0
5,1943,1943.0,1943.0,1943.0,1943.0,1943.0,1952.0,1952.0,1943.0,1943.0,...,2014-03-17 15:22:43,2014-03-17 15:20:10,2014-03-17 15:21:40,2014-03-17 15:22:10,2014-03-17 15:22:39,2014-03-17 15:22:39,2014-03-17 15:22:41,2014-03-17 15:22:41,2014-03-17 15:22:42,1737.0


В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95319 entries, 1 to 95319
Data columns (total 21 columns):
site1      95319 non-null int64
time1      95319 non-null object
site2      93722 non-null float64
time2      93722 non-null object
site3      92339 non-null float64
time3      92339 non-null object
site4      91085 non-null float64
time4      91085 non-null object
site5      89868 non-null float64
time5      89868 non-null object
site6      88776 non-null float64
time6      88776 non-null object
site7      87755 non-null float64
time7      87755 non-null object
site8      86738 non-null float64
time8      86738 non-null object
site9      85754 non-null float64
time9      85754 non-null object
site10     84810 non-null float64
time10     84810 non-null object
user_id    95319 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 16.0+ MB


In [7]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,1917,2014-12-04 20:36:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20225,2014-03-04 14:08:55,27.0,2014-03-04 14:08:59,9.0,2014-03-04 14:08:59,32.0,2014-03-04 14:09:50,20225.0,2014-03-04 14:10:43,20258.0,2014-03-04 14:10:44,20225.0,2014-03-04 14:10:45,20225.0,2014-03-04 14:11:07,20276.0,2014-03-04 14:11:08,20225.0,2014-03-04 14:11:09
3,71,2014-03-31 09:20:38,63.0,2014-03-31 09:20:42,64.0,2014-03-31 09:20:43,63.0,2014-03-31 09:20:43,71.0,2014-03-31 09:20:43,22.0,2014-03-31 09:20:45,64.0,2014-03-31 09:20:46,71.0,2014-03-31 09:20:47,71.0,2014-03-31 09:20:48,70.0,2014-03-31 09:20:51
4,7338,2014-02-14 13:39:36,7338.0,2014-02-14 14:07:53,7338.0,2014-02-14 14:08:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,19622,2014-03-31 09:46:04,32.0,2014-03-31 09:46:12,340.0,2014-03-31 09:46:13,19634.0,2014-03-31 09:46:14,1721.0,2014-03-31 09:46:14,327.0,2014-03-31 09:46:15,38.0,2014-03-31 09:46:19,1721.0,2014-03-31 09:46:19,340.0,2014-03-31 09:46:20,19634.0,2014-03-31 09:46:20


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41177 entries, 1 to 41177
Data columns (total 20 columns):
site1     41177 non-null int64
time1     41177 non-null object
site2     39529 non-null float64
time2     39529 non-null object
site3     38158 non-null float64
time3     38158 non-null object
site4     37030 non-null float64
time4     37030 non-null object
site5     36049 non-null float64
time5     36049 non-null object
site6     35083 non-null float64
time6     35083 non-null object
site7     34284 non-null float64
time7     34284 non-null object
site8     33434 non-null float64
time8     33434 non-null object
site9     32633 non-null float64
time9     32633 non-null object
site10    31907 non-null float64
time10    31907 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 6.6+ MB


**В обучающей выборке – 550 пользователей.**

In [9]:
train_df['user_id'].nunique()

550

In [10]:
train_df['user_id'].value_counts().head()

3165    3351
537     1927
3324    1534
1826    1528
1845    1452
Name: user_id, dtype: int64

**Пока для прогноза ID пользователя будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [11]:
train_test_df_sites = train_test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype('int')

In [12]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,8,11,82,68,8,8393,8403,932,3260,8
2,111,78,151,111,1401,151,1375,38,1401,97
3,11,3187,132,496,1969,504,3187,82,3191,3184
4,668,1965,598,1965,284,668,38,4451,4537,11
5,1943,1943,1943,1943,1943,1952,1952,1943,1943,1943
6,2941,2951,2958,2993,162,2967,85,7,2967,2967
7,9996,9996,307,9996,280,9996,9996,307,9996,9996
8,19823,19823,1510,32,19808,8,567,567,654,567
9,72,0,0,0,0,0,0,0,0,0


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу train_test_df_sites – потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [13]:
def make_sparse_data(data):
    indptr = [0]
    indices = []
    sparse_data = []
    for row in data:
        val, cnt = np.unique(row[row != 0], return_counts=True)
        indptr.append(indptr[-1] + len(val))
        for v, c in zip(val, cnt):
            indices.append(v - 1)
            sparse_data.append(c)
    return np.uint64(sparse_data),  np.uint64(indices), np.uint64(indptr)

In [14]:
col_size = np.unique(train_test_df_sites.values).shape[0] - 1
row_size_full = train_test_df_sites.shape[0]

In [15]:
X_train_test_sparse = csr_matrix(make_sparse_data(train_test_df_sites.values))
X_train_sparse = csr_matrix(make_sparse_data(train_test_df_sites.values[:95319]), shape=(95319, col_size))
X_test_sparse = csr_matrix(make_sparse_data(train_test_df_sites.values[95319:]), shape=(row_size_full - 95319, col_size))
#y = csr_matrix(make_sparse_data(train_df.user_id.values))
y = train_df.user_id

In [17]:
X_train_test_sparse.shape

(136496, 24052)

In [18]:
answer = str(X_train_sparse.shape[0]) + ' ' + str(X_train_sparse.shape[1]) + ' ' + \
         str(X_test_sparse.shape[0]) + ' ' + str(X_test_sparse.shape[1])
answer

'95319 24052 41177 24052'

In [19]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

**Запишите в файл *answer5_1.txt* размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*. Полученный файл будет ответом на 1 вопрос теста.**

In [20]:
write_answer_to_file(answer, 'answer5_1.txt')

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [22]:
with open('kaggle_data/X_train_sparse.pkl', 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl)
with open('kaggle_data/X_test_sparse.pkl', 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl)
with open('kaggle_data/train_target.pkl', 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl)

**Разобьем обучающую выборку на 2 части в пропорции 7/3.**

In [105]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, test_size=0.3, 
                                                     random_state=7, stratify=y)

**Создайте объекты sklearn.linear_model.SGDClassifier с логистической функцией потерь и с hinge loss (логистическая регрессия и линейный SVM соответственно) и параметром random_state=7. Остальные параметры оставьте по умолчанию, разве что n_jobs=-1 никогда не помешает. Обучите  модели на выборке (X_train, y_train).**

In [110]:
sgd_logit = SGDClassifier(loss='log', random_state=7)

In [111]:
%%time
sgd_logit.fit(X_train, y_train)

Wall time: 41.1 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=7, shuffle=True, verbose=0,
       warm_start=False)

In [112]:
sgd_svm = SGDClassifier(loss='hinge', random_state=7)

In [113]:
%%time
sgd_svm.fit(X_train, y_train)

Wall time: 25.6 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=7, shuffle=True, verbose=0,
       warm_start=False)

**Сделаем прогнозы с помощью обеих моделей на отложенной выборке (X_valid, y_valid).**

In [114]:
logit_valid_pred = sgd_logit.predict(X_valid)
svm_valid_pred = sgd_svm.predict(X_valid)

In [115]:
print(accuracy_score(y_valid, logit_valid_pred))
print(accuracy_score(y_valid, svm_valid_pred))

0.286403692824
0.267135263673


**Запишите в файл *answer5_2.txt* через пробел доли правильных ответов логистической регресии и линейного SVM, обученных с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя. Полученный файл будет ответом на 2 вопрос теста.**

In [117]:
write_answer_to_file("{} {}".format(round(0.286403692824,3), round(0.267135263673,3)) ,
                     'answer5_2.txt')

**Сделайте прогноз для тестовой выборки с помощью sgd_logit.**

In [120]:
logit_test_pred = sgd_logit.predict(X_test_sparse)

**Запишите ответы в файл и сделайте посылку на Kaggle. Далее будет короткое Peer-Review задание, в котором надо проверить посылки друг друга на Kaggle. Поэтому дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera Username", чтоб можно было идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/leaderboard).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить, дополнительные баллы будут для тех, кто попадет в топ-10 и топ-30 по итогам этой недели.**

In [121]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [122]:
write_to_submission_file(logit_test_pred, "./kaggle_data/predictions.csv")

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в финальный проект (.pdf или .ipynb).
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных 550 пользователями веб-страницах (550 csv-файлов в *train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем тьюториал по Vowpal Wabbit и попробуем его в деле – в соревновании.